In [3]:
NAME_DB = 'dc3el4uteo7v6u'
HOST_DB = "ec2-34-242-154-118.eu-west-1.compute.amazonaws.com"
PORT_DB = "5432"
USER_DB = "jvsrubhyybbrqv"
PASSWORD_DB = "9678992a26f6926e72b504835122a0a39b08c221c2121cd98a65b094da595a9d"

config_ebay_keywords = {
    '1':'A game of thrones first edition'
}

config_edition_model = {
    '1' :'2_AGOT_Model_2024-02-27.h5'
}

ML_MODELS_DIRECTORY = '/content/drive/My Drive/Tech/Production/ML_Models/'

app_id = 'MarcoFra-BBourse-PRD-5e5603d2f-045f9a67'

In [ ]:
config_edition_ml = {
    '1':{
        'ebay_api_keywords':'A game of thrones first edition',
        'ml_model' :'2_AGOT_Model_2024-02-27.h5'
     }
}

In [4]:
import pandas as pd
from sqlalchemy import create_engine
import requests

In [6]:
DATABASE_URL = f"postgresql://{USER_DB}:{PASSWORD_DB}@{HOST_DB}:{PORT_DB}/{NAME_DB}"
engine = create_engine(DATABASE_URL)

In [7]:
def return_ml_books(edition_id):

  query = f"SELECT * FROM bbourse.ml_books WHERE 1=1 and edition_id = {edition_id}"
  df = pd.read_sql(query, engine)

  return df

In [8]:
def find_active_items(app_id, edition_id, total_entries=1000):
    url = 'https://svcs.ebay.com/services/search/FindingService/v1'
    entries_per_page = 100
    items = []

    for page_number in range(1, (total_entries // entries_per_page) + 2):
        params = {
            'OPERATION-NAME': 'findItemsByKeywords',
            'SERVICE-VERSION': '1.13.0',
            'SECURITY-APPNAME': app_id,
            'GLOBAL-ID': 'EBAY-US',
            'RESPONSE-DATA-FORMAT': 'JSON',
            'keywords': config_ebay_keywords[edition_id],
            'paginationInput.entriesPerPage': entries_per_page,
            'paginationInput.pageNumber': page_number,
            'sortOrder': 'EndTimeSoonest',
        }

        response = requests.get(url, params=params)
        if response.status_code == 200:
            json_response = response.json()
            search_result = json_response.get('findItemsByKeywordsResponse', [])[0].get('searchResult', [{}])[0]
            items_on_page = search_result.get('item', [])
            items.extend(items_on_page)
            # Break if we have collected all items or if there are no more items to collect
            if len(items_on_page) < entries_per_page or len(items) >= total_entries:
                break
        else:
            print(f"Error on page {page_number}: {response.status_code}")
            break

    return items[:total_entries]



def ebay_listings(app_id, edition_id ):
  active_items = find_active_items(app_id, edition_id, 1000)
  df = pd.DataFrame(data=active_items)

  df_small = df[['itemId','title','galleryURL','viewItemURL','sellingStatus']]
  df_small = df_small.applymap(lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x)

  def extract_converted_price(row):
      return row['convertedCurrentPrice'][0]['__value__']

  df_small['price'] = df_small['sellingStatus'].apply(extract_converted_price)


  df_small = df_small[['itemId','title','galleryURL','viewItemURL','price']]
  return df_small


In [9]:
def retrieve_new_ebay_listings(app_id, edition_id):
  df_ebay = ebay_listings(app_id, "1")
  
  df_db = return_ml_books(edition_id)
  df_total = pd.merge(df_ebay, df_db['img_link'], left_on ='galleryURL', right_on ='img_link', how='left')
  df_total = df_total[df_total['img_link'].isna()]
  df_total['edition_id'] = edition_id
  df_total.drop(['img_link'], axis=1, inplace=True)
  return df_total

In [10]:
df = retrieve_new_ebay_listings(app_id, "1")
df

C:\Users\marco\AppData\Local\Temp\ipykernel_17688\912750697.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_small = df_small.applymap(lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x)


,itemId,title,galleryURL,viewItemURL,price,edition_id
31,256489278314,A Game Of Thrones 1997 Paperback Blue/Grey Jon...,https://i.ebayimg.com/thumbs/images/g/-4kAAOSw...,https://www.ebay.com/itm/Game-Thrones-1997-Pap...,6.0,1
66,256490583751,A Game Of Thrones Jon Snow Original Paperback ...,https://i.ebayimg.com/thumbs/images/g/j8kAAOSw...,https://www.ebay.com/itm/Game-Thrones-Jon-Snow...,5.87,1
214,196364355715,GEORGE R R MARTIN ** SIGNED ** ARC TRUE FIRST ...,https://i.ebayimg.com/thumbs/images/g/Ni4AAOSw...,https://www.ebay.com/itm/GEORGE-R-R-MARTIN-SIG...,10000.0,1
289,364341163246,GEORGE R.R.MARTIN A Clash Of Kings Bantam HC R...,https://i.ebayimg.com/thumbs/images/g/Q6YAAOSw...,https://www.ebay.com/itm/GEORGE-R-R-MARTIN-Cla...,59.95,1
471,375380597114,The Identicals : A Novel by Elin Hilderbrand 2...,https://i.ebayimg.com/thumbs/images/g/zUEAAOSw...,https://www.ebay.com/itm/Identicals-Novel-Elin...,16.99,1
721,156179677925,"Fire & Blood - George R.R. Martin (2018, HCDJ)...",https://i.ebayimg.com/thumbs/images/g/MxsAAOSw...,https://www.ebay.com/itm/Fire-Blood-George-R-R...,74.99,1
747,315318974410,A Storm of Swords * A Game of Thrones Book 3 *...,https://i.ebayimg.com/thumbs/images/g/lgoAAOSw...,https://www.ebay.com/itm/Storm-Swords-Game-Thr...,14.95,1
755,386956154841,Game Of Thrones 1996 George RR Martin 1st Ed F...,https://i.ebayimg.com/thumbs/images/g/~MEAAOSw...,https://www.ebay.com/itm/Game-Thrones-1996-Geo...,350.0,1
762,135033447794,"FFG Game of Thrones CCG Game of Thrones, A - T...",https://i.ebayimg.com/thumbs/images/g/Q5kAAOSw...,https://www.ebay.com/itm/FFG-Game-Thrones-CCG-...,9.0,1
763,355665519053,Book In English. A Song Of Ice And Fire. Book ...,https://i.ebayimg.com/thumbs/images/g/xoUAAOSw...,https://www.ebay.com/itm/Book-English-Song-Ice...,65.0,1
